In [1]:
import zipfile 
import pysal as ps
%matplotlib inline
import numpy as np
import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import requests
import os
import geopandas as gpd
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import matplotlib.pylab as pl
from matplotlib import colors
import shapely
import seaborn as sns
import gzip
import mapreduce as mr
import csv

In [2]:
fname = 'Data/CensusTracks.geojson'
tracks = gpd.GeoDataFrame.from_file(fname)
validCT = set(tracks[tracks.Within30min == True].boro_ct_2010)

In [41]:
def maptracks(row):
    if row['h_track'] in validCT:
        if row['h_track'][0]=='3':
            #print 'yes'
            if row['w_track'][0]=='1':
                #print 'yes'
                yield (row['w_track'], row['h_track'],int(row['S000']))

def reducehtracks(counts, pair_int):
    pair=pair_int[0:2]
    counts[pair] = counts.get(pair, 0)+pair_int[-1]
    return counts

with open('Data/True_nyod_JT00thegood_2014.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output2 = sorted(reduce(reducehtracks,list(mr.run(reader,maptracks)), {}).items())
    #
print len(output2)
output2[:10]

28995


[(('1000100', '3029700'), 2),
 (('1000100', '3030700'), 1),
 (('1000100', '3037700'), 1),
 (('1000100', '3038100'), 2),
 (('1000100', '3089400'), 1),
 (('1000100', '3090000'), 3),
 (('1000100', '3098200'), 1),
 (('1000100', '3098400'), 3),
 (('1000100', '3113000'), 2),
 (('1000100', '3114400'), 1)]

In [36]:
def maptracks2(row):
    yield row[0][0], row[0][1], row[1],


def reducehtracks(pair_int):
 
    return pair_int[0],pair_int[1],pair_int[2],

output3 = list(mr.run(output2,maptracks2))#,reducehtracks))
    #
print len(output3)
output3[:10]

28995


[('1000100', '3029700', 2),
 ('1000100', '3030700', 1),
 ('1000100', '3037700', 1),
 ('1000100', '3038100', 2),
 ('1000100', '3089400', 1),
 ('1000100', '3090000', 3),
 ('1000100', '3098200', 1),
 ('1000100', '3098400', 3),
 ('1000100', '3113000', 2),
 ('1000100', '3114400', 1)]

In [53]:
def maptracks2(both_int):
    yield both_int[0][0], both_int[0][1], both_int[1],


def reducehtracks(work_home_int):
    tupl= work_home_int[1],work_home_int[1],
 
    return work_home_int[0]

output4 = list(mr.run(output2,maptracks2,reducehtracks))
    #
print len(output4)
output4[:10]

285


['1000100',
 '1000201',
 '1000202',
 '1000500',
 '1000600',
 '1000700',
 '1000800',
 '1000900',
 '1001001',
 '1001002']

In [55]:
def maptracks2(both_int):
    yield both_int[0][0], both_int[0][1], both_int[1]


def reducehtracks3(work_home_int):
 
    return work_home_int[1],work_home_int[1]

output5 = list(mr.run(output2,maptracks2))
    #
print len(output5)
output5[:10]

28995


[('1000100', '3029700', 2),
 ('1000100', '3030700', 1),
 ('1000100', '3037700', 1),
 ('1000100', '3038100', 2),
 ('1000100', '3089400', 1),
 ('1000100', '3090000', 3),
 ('1000100', '3098200', 1),
 ('1000100', '3098400', 3),
 ('1000100', '3113000', 2),
 ('1000100', '3114400', 1)]

In [57]:
def maptracks2(w_h_int):
    yield w_h_int[1]
    
output6 = sorted(set(mr.run(output5,maptracks2)))
print len(output6)
output6[:10]

239


['3000100',
 '3002100',
 '3019100',
 '3024100',
 '3025300',
 '3025500',
 '3025700',
 '3025901',
 '3025902',
 '3026100']

In [58]:
homes=output6

In [33]:
jobs = output4

In [174]:
lodes = gpd.GeoDataFrame(columns=homes, index=jobs)

In [175]:
def maptodf(w_h_int):
    lodes.loc[w_h_int[0],w_h_int[1]]=w_h_int[2]
    
    return None

map(maptodf,output5)
print('done')

done


In [176]:
lodes.reset_index(inplace=True)
lodes.rename(inplace=True,columns={'index':'boro_ct_2010'})
lodes.head()


,boro_ct_2010,3000100,3002100,3019100,3024100,3025300,3025500,3025700,3025901,3025902,...,3117400,3117601,3117800,3118000,3119000,3119200,3119400,3119600,3119800,3123700
0,1000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000201,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000202,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
3,1000500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000600,2,NaN,NaN,NaN,NaN,1,1,NaN,NaN,...,2,NaN,NaN,1,NaN,1,NaN,1,NaN,2


In [179]:
lodes.shape

(285, 240)

In [180]:
lodes.describe()

,boro_ct_2010,3000100,3002100,3019100,3024100,3025300,3025500,3025700,3025901,3025902,...,3117400,3117601,3117800,3118000,3119000,3119200,3119400,3119600,3119800,3123700
count,285,146,146,133,118,129,139,118,107,121,...,139,109,97,35,105,107,119,148,112,119
unique,285,32,40,22,21,21,23,16,16,17,...,19,11,14,3,13,14,20,24,16,15
top,1016300,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
freq,1,36,40,44,36,37,41,47,41,50,...,51,44,46,33,40,42,48,55,45,56


In [181]:
tracks.head(3)

,Within15min,Within30min,boro_code,boro_ct_2010,boro_name,cdeligibil,ct_2010,ctlabel,geometry,id,ntacode,ntaname,puma,shape_area,shape_leng
0,0,0,5,5000900,Staten Island,I,000900,9,(POLYGON ((-74.07920577013245 40.6434307837456...,0,SI22,West New Brighton-New Brighton-St. George,3903,2497009.69813,7729.01679376
1,0,0,5,5007400,Staten Island,I,007400,74,(POLYGON ((-74.05974734759452 40.5938486115672...,1,SI14,Grasmere-Arrochar-Ft. Wadsworth,3902,5788237.79601,9902.94847281
2,0,0,1,1009800,Manhattan,I,009800,98,(POLYGON ((-73.96432543478758 40.7563815309909...,2,MN19,Turtle Bay-East Midtown,3808,1906016.35002,5534.19981063


In [182]:
tojoin = tracks[['geometry','boro_ct_2010']]
tojoin.head()

,geometry,boro_ct_2010
0,(POLYGON ((-74.07920577013245 40.6434307837456...,5000900
1,(POLYGON ((-74.05974734759452 40.5938486115672...,5007400
2,(POLYGON ((-73.96432543478758 40.7563815309909...,1009800
3,(POLYGON ((-73.96802436915851 40.7595781400528...,1010000
4,(POLYGON ((-73.97124277307127 40.7609364184790...,1010200


In [183]:
lodes = lodes.merge(tojoin, on='boro_ct_2010', how='left')

In [184]:
lodes.head(3)

,boro_ct_2010,3000100,3002100,3019100,3024100,3025300,3025500,3025700,3025901,3025902,...,3117601,3117800,3118000,3119000,3119200,3119400,3119600,3119800,3123700,geometry
0,1000100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(POLYGON ((-74.0438776163991 40.69018767537123...
1,1000201,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(POLYGON ((-73.98449647076764 40.7095096639394...
2,1000202,1,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,(POLYGON ((-73.98237340572007 40.7055433504376...


In [186]:
lodes.to_file('Data/affectedjobs_homesmatrix.geojson')

In [187]:
lodesT = lodes.T
lodesT.head()

,0,1,2,3,4,5,6,7,8,9,...,275,276,277,278,279,280,281,282,283,284
boro_ct_2010,1000100,1000201,1000202,1000500,1000600,1000700,1000800,1000900,1001001,1001002,...,1029300,1029500,1029700,1029900,1030300,1030700,1030900,1031703,1031704,1031900
3000100,NaN,1,1,NaN,2,45,1,34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,2
3002100,NaN,NaN,NaN,NaN,NaN,45,1,48,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,1
3019100,NaN,NaN,NaN,NaN,NaN,20,1,24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3024100,NaN,NaN,NaN,NaN,NaN,8,NaN,23,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
lodesT.columns = lodesT.iloc[0]

In [189]:
lodesT.head(3)

boro_ct_2010,1000100,1000201,1000202,1000500,1000600,1000700,1000800,1000900,1001001,1001002,...,1029300,1029500,1029700,1029900,1030300,1030700,1030900,1031703,1031704,1031900
boro_ct_2010,1000100,1000201,1000202,1000500,1000600,1000700,1000800,1000900,1001001,1001002,...,1029300,1029500,1029700,1029900,1030300,1030700,1030900,1031703,1031704,1031900
3000100,NaN,1,1,NaN,2,45,1,34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,2
3002100,NaN,NaN,NaN,NaN,NaN,45,1,48,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,1


In [190]:
lodesT = lodesT.iloc[1:-1,:]

In [191]:
lodesT.reset_index(inplace=True)
lodesT.rename(inplace=True,columns={'index':'boro_ct_2010'})
lodesT.head()

boro_ct_2010,boro_ct_2010,1000100,1000201,1000202,1000500,1000600,1000700,1000800,1000900,1001001,...,1029300,1029500,1029700,1029900,1030300,1030700,1030900,1031703,1031704,1031900
0,3000100,NaN,1,1,NaN,2,45,1,34,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,2
1,3002100,NaN,NaN,NaN,NaN,NaN,45,1,48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,1
2,3019100,NaN,NaN,NaN,NaN,NaN,20,1,24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3024100,NaN,NaN,NaN,NaN,NaN,8,NaN,23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3025300,NaN,1,NaN,NaN,NaN,13,1,11,NaN,...,NaN,NaN,NaN,2,NaN,NaN,NaN,1,2,NaN


In [192]:
lodesT.index.rename('', inplace=True)

In [193]:
lodesT.head()

boro_ct_2010,boro_ct_2010,1000100,1000201,1000202,1000500,1000600,1000700,1000800,1000900,1001001,...,1029300,1029500,1029700,1029900,1030300,1030700,1030900,1031703,1031704,1031900
,,,,,,,,,,,,,,,,,,,,,
0,3000100,NaN,1,1,NaN,2,45,1,34,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,2
1,3002100,NaN,NaN,NaN,NaN,NaN,45,1,48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,1
2,3019100,NaN,NaN,NaN,NaN,NaN,20,1,24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN
3,3024100,NaN,NaN,NaN,NaN,NaN,8,NaN,23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3025300,NaN,1,NaN,NaN,NaN,13,1,11,NaN,...,NaN,NaN,NaN,2,NaN,NaN,NaN,1,2,NaN


In [194]:
lodesT = lodesT.merge(tojoin, on='boro_ct_2010', how='left')

In [195]:
lodesT.head()

boro_ct_2010,boro_ct_2010,1000100,1000201,1000202,1000500,1000600,1000700,1000800,1000900,1001001,...,1029500,1029700,1029900,1030300,1030700,1030900,1031703,1031704,1031900,geometry
0,3000100,NaN,1,1,NaN,2,45,1,34,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,2,(POLYGON ((-73.99093201741492 40.7007662160486...
1,3002100,NaN,NaN,NaN,NaN,NaN,45,1,48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,1,(POLYGON ((-73.97905767012279 40.7059528891005...
2,3019100,NaN,NaN,NaN,NaN,NaN,20,1,24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,(POLYGON ((-73.96015854658333 40.6941173091560...
3,3024100,NaN,NaN,NaN,NaN,NaN,8,NaN,23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(POLYGON ((-73.95146465503193 40.6898633417958...
4,3025300,NaN,1,NaN,NaN,NaN,13,1,11,NaN,...,NaN,NaN,2,NaN,NaN,NaN,1,2,NaN,(POLYGON ((-73.94891392355648 40.6939048667599...


In [197]:
lodesT.to_file('Data/affectedhomes_jobsmatrix.geojson')